In [ ]:
import torch
import numpy as np

#check cholesky works as intended with some random matrix

mat = np.random.normal(0,1,[5,5])
mat = np.dot(mat, mat.T)
chol_numpy = np.linalg.cholesky(mat)
print(chol_numpy)
chol_torch = torch.cholesky(torch.tensor(mat))
print(chol_torch)
chol_torch_numpy = chol_torch.numpy()
print(chol_torch_numpy==chol_numpy)
print(torch.t(chol_torch))
#okay, so for all intents and purposes they are the same, minus some numerical matters!

[[ 2.65105209  0.          0.          0.          0.        ]
 [-0.08286265  2.57499571  0.          0.          0.        ]
 [ 0.82753411  1.19137239  1.40638461  0.          0.        ]
 [ 0.48376595 -1.07474231  0.94137085  0.76101544  0.        ]
 [ 1.33800368 -0.85891361  0.23952361  1.57026559  0.95257107]]
tensor([[ 2.6511,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0829,  2.5750,  0.0000,  0.0000,  0.0000],
        [ 0.8275,  1.1914,  1.4064,  0.0000,  0.0000],
        [ 0.4838, -1.0747,  0.9414,  0.7610,  0.0000],
        [ 1.3380, -0.8589,  0.2395,  1.5703,  0.9526]], dtype=torch.float64)
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True False  True]
 [ True  True  True False False]]
tensor([[ 2.6511, -0.0829,  0.8275,  0.4838,  1.3380],
        [ 0.0000,  2.5750,  1.1914, -1.0747, -0.8589],
        [ 0.0000,  0.0000,  1.4064,  0.9414,  0.2395],
        [ 0.0000,  0.0000,  0.0000,  0.7610,  1.5703],
 

In [ ]:
# implement basic unscented transform
vec = np.random.normal(0,1,[5,1])
print(torch.tensor(vec).squeeze(1).shape)
print(vec)
print(torch.diag(torch.tensor(vec).squeeze(1)))

torch.Size([5])
[[ 0.26573669]
 [-0.96489474]
 [-0.43928972]
 [ 0.97369676]
 [-0.6375404 ]]
tensor([[ 0.2657,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.9649,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.4393,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.9737,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, -0.6375]], dtype=torch.float64)


In [ ]:
#beginning sigma points
dim = 2
alpha = 0.1
beta = 2
kappa = 1

#compute unscented weights
lambda_ = alpha**2 * (dim+kappa) - dim
Wc = torch.tensor([1]).new_full([2*dim+1], 1./(2*(dim + lambda_)))
Wm = torch.tensor([1]).new_full([2*dim+1], 1./(2*(dim + lambda_)))
Wc[0] = lambda_/(dim+lambda_) + 1 - alpha**2 + beta
Wm[0] = lambda_/(dim + lambda_)
print(Wc)
print(Wm)

#implement sigma points
mu = torch.randn([dim,])
var = torch.abs(torch.randn([dim,1]))
sigmas = torch.zeros([2*dim+1,dim])
#get past covariance #not sure how to change it to log space
P = torch.diag(var.squeeze(1))
U = torch.t(torch.cholesky(P))
print(P)
print(U)
print(sigmas.shape)
print(mu.shape)
print(mu) #upper triangular cholesky factorization
sigmas[0,:] = mu #the means
for k in range(dim):
  sigmas[k+1,:] = mu + U[k]
  sigmas[dim+k+1,:] = mu - U[k]

#newmean
print(sigmas)
print(Wm.shape)
print(sigmas.shape)
#do it by hand
kmax, n = sigmas.shape
new_mu = torch.zeros([2,])
print(a.shape)
for i in range(kmax):
  print(Wm[i].shape)
  print(sigmas[i,:].shape)
  new_mu += Wm[i] * sigmas[i,:]
print(new_mu)

new_P = torch.zeros([n,n])
for k in range(kmax):
  y = sigmas[k,:] - new_mu
  new_P += Wc[k] * torch.ger(y,y)
print(new_P)
#that should obtain the sigma points correctly right
#then we just need to get the new mus
#no it gives a diagonal covariance apparently... which makes sense
#since the initial P is diagonal


tensor([-62,  16,  16,  16,  16])
tensor([-65,  16,  16,  16,  16])
tensor([[1.8416, 0.0000],
        [0.0000, 1.1254]])
tensor([[1.3570, 0.0000],
        [0.0000, 1.0609]])
torch.Size([5, 2])
torch.Size([2])
tensor([ 0.2085, -0.4999])
tensor([[ 0.2085, -0.4999],
        [ 1.5656, -0.4999],
        [ 0.2085,  0.5609],
        [-1.1485, -0.4999],
        [ 0.2085, -1.5608]])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
torch.Size([])
torch.Size([2])
torch.Size([])
torch.Size([2])
torch.Size([])
torch.Size([2])
torch.Size([])
torch.Size([2])
torch.Size([])
torch.Size([2])
tensor([-0.2085,  0.4999])
tensor([[58.9300,  0.0000],
        [ 0.0000, 36.0137]])
